In [1]:
import pandas as pd
df=pd.read_csv("E:/nbn/New folder (2)/dataset-sts-master/data/anssel/yodaqa/large2470-training.csv")
df=df[:20000]
df.head()

,qtext,label,atext,kwweight,aboutkwweight,toklabels
0,What is the street address of the White House ?,1,The North Lawn at the White House in Washingto...,0.0,2.3,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 ...
1,What is the street address of the White House ?,1,Because it is bordered by Pennsylvania Avenue ...,5.6,2.3,0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
2,What is the street address of the White House ?,0,A circular pool with fountain is centered on t...,0.0,2.3,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
3,What is the street address of the White House ?,0,Visiting heads of state enter the White House ...,0.0,2.3,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
4,What is the street address of the White House ?,0,"White House correspondents , for television ne...",0.0,2.3,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


In [2]:
import nltk
from nltk.corpus import stopwords
import nltk.data
from nltk.stem import PorterStemmer
from nltk.util import ngrams
import pandas as pd
import string
from string import punctuation
stopWords=set(stopwords.words('english'))
punc=list(string.punctuation)


In [3]:
df=df.fillna(' ')

In [4]:
def remove_stopWords(text):#
    t=[]
    for word in text.split(): #tokens nltk.wordtoknize(text)
        #stripe() =>removes any spaces or specified characters at the start and end of a string
        if word.strip().lower() not in stopWords:
            t.append(word)
    return " ".join(t)
#remove punctuation
def remove_punctuation(text):
    t=[]
    for word in text.split(): #tokens
        #stripe() =>removes any spaces or specified characters at the start and end of a string
        if word.strip().lower() not in punc:
            t.append(word)
            
    return " ".join(t)

In [5]:
def limma(text):
    t=[]
    wnl=nltk.WordNetLemmatizer()
    for word in text.split():
        if word.strip() not in stopWords and word not in punc:
            t.append(wnl.lemmatize(word))
    return " ".join(t)


In [6]:
del df['toklabels']
del df['aboutkwweight']
del df['kwweight']

In [7]:
df['qtext']=df['qtext'].apply(remove_stopWords)
df['qtext']=df['qtext'].apply(remove_punctuation)
df['qtext']=df['qtext'].apply(limma)

In [8]:
df['atext']=df['atext'].apply(remove_stopWords)
df['atext']=df['atext'].apply(remove_punctuation)
df['atext']=df['atext'].apply(limma)

In [9]:
df['text']=df['qtext']+" "+df['atext']
df.head()

,qtext,label,atext,text
0,street address White House,1,North Lawn White House Washington DC bordered ...,street address White House North Lawn White Ho...
1,street address White House,1,bordered Pennsylvania Avenue White House 's of...,street address White House bordered Pennsylvan...
2,street address White House,0,circular pool fountain centered north portico ...,street address White House circular pool fount...
3,street address White House,0,Visiting head state enter White House ground o...,street address White House Visiting head state...
4,street address White House,0,White House correspondent television news ofte...,street address White House White House corresp...


In [10]:
y=df.iloc[:,1].values

In [11]:
df.head()

,qtext,label,atext,text
0,street address White House,1,North Lawn White House Washington DC bordered ...,street address White House North Lawn White Ho...
1,street address White House,1,bordered Pennsylvania Avenue White House 's of...,street address White House bordered Pennsylvan...
2,street address White House,0,circular pool fountain centered north portico ...,street address White House circular pool fount...
3,street address White House,0,Visiting head state enter White House ground o...,street address White House Visiting head state...
4,street address White House,0,White House correspondent television news ofte...,street address White House White House corresp...


In [12]:
del df['label']

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
qvec=TfidfVectorizer()
qfeatures=qvec.fit_transform(df['text']).toarray()

In [14]:
s=sorted(qvec.vocabulary_.keys())
print(len(s))
print(len(qfeatures[0]))

30710
30710


In [15]:
text=[]
for i in range(len(df)):
    result=[0]*len(s)
    for word in df['qtext'][i].split():
        if word in s:
            result[s.index(word)] = qfeatures[i][s.index(word)]
    text.append(result)
                

In [16]:
for i in text[0]:
    if i != 0:
        print(i)

0.12562020495092815
0.12453544022125125


In [17]:
atext=[]
for i in range(len(df)):
    result=[0]*len(s)
    for word in df['atext'][i].split():
        if word in s:
            result[s.index(word)] = qfeatures[i][s.index(word)]
    atext.append(result)
                

In [18]:
len(atext[0])

30710

0.1600904205748002
0.24190704934440824
0.21861698699543586
0.24190704934440824
0.12295664448035452
0.1743782270492314
0.19155441985863367
0.1933707470009295
0.411941597668742
0.1860585117343135
0.16220979470289607
0.22428879730857654
0.2492192846110242
0.2276828668452281
0.21398278053936085
0.15860935517054997
0.1933707470009295


In [21]:
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances
import numpy as np
text=np.array(text)
atext=np.array(atext)
cofeature=[]
for i in range(len(text)):
    cofeature.append(cosine_similarity(text[i].reshape(1,-1),atext[i].reshape(1,-1)))

In [22]:
x=[]
for i in range(len(cofeature)):
    x.append(cofeature[i][0][0])

In [23]:

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [24]:
x_train=np.array(x_train).reshape(-1,1)
x_test=np.array(x_test).reshape(-1,1)

print(x_train.shape)
print(x_test.shape)

(16000, 1)
(4000, 1)


In [25]:
from sklearn.naive_bayes import MultinomialNB
#use the classifier
model= MultinomialNB()
model.fit(x_train,y_train)

MultinomialNB()

In [26]:
predicted=model.predict(x_test)
print(np.mean(predicted==y_test))

0.9245


,qtext,atext,text
0,street address White House,North Lawn White House Washington DC bordered ...,street address White House North Lawn White Ho...
1,street address White House,bordered Pennsylvania Avenue White House 's of...,street address White House bordered Pennsylvan...
2,street address White House,circular pool fountain centered north portico ...,street address White House circular pool fount...
3,street address White House,Visiting head state enter White House ground o...,street address White House Visiting head state...
4,street address White House,White House correspondent television news ofte...,street address White House White House corresp...
